In [1]:
import pandas as pd
import numpy as np
from glob import glob
import datetime,time
from dateutil.relativedelta import relativedelta , TH
import warnings
warnings.filterwarnings('ignore')

In [2]:
path = pd.DataFrame(glob('C:\sample_data_Jan2019/*'),columns = ["Location"])

In [3]:
path = pd.DataFrame(glob('C:\sample_data_Jan2019/*') , columns = ['location'])
path['datetime'] = path['location'].apply(lambda x: x.split('_')[-1].split('.')[0])
path['datetime']= pd.to_datetime(path['datetime'])

In [4]:
trade_log = pd.DataFrame(columns = ['Entry_date_time' , 'Day_to_exp' , 'entry_ce_price_1' , 'entry_pe_price_1' , 'entry_ce_strike_price_1' ,
                                   'entry_pe_strike_price_1' , 'ce_ltp' , 'pe_ltp' , 'ce_pnl' , 'pe_pnl' , 'ce_exit_time' , 'put_exit_time' , 'pnl'])

In [5]:
for index,row in path.iterrows():
    print(index)
    data = pd.read_pickle(row['location'])
    entry_date_time=datetime.datetime.combine(row['datetime'].date(),datetime.time(9,16))
    exit_date_time=datetime.datetime.combine(row['datetime'].date(),datetime.time(15,20))
    do_not_entry = datetime.datetime.combine(row['datetime'].date(),datetime.time(14,59))
    data_bn =data[(data['instrument_name']=='BANKNIFTY')].reset_index(drop = 'Frist')

    data_bn=data_bn[~data_bn['instrument_type'].isin(['FUT'])]
    data_bn_1 =data_bn[data_bn['datetime'] >= entry_date_time].reset_index(drop='Frist')
    nearest_exp = row['datetime'].date()+relativedelta(weekday = TH)
    data_ce = data_bn[(data_bn['instrument_type'] == 'CE') & (data_bn['expiry_date'] ==nearest_exp)|(data_bn['expiry_date'] == nearest_exp - datetime.timedelta(days=1))|(data_bn['expiry_date'] == nearest_exp - datetime.timedelta(days=2)) &  (data_bn['datetime'] > entry_date_time)].reset_index(drop='Frist')
    entry_ce_price_1 = data_ce[(data_ce['open'] < 65) & (data_ce['datetime'] == entry_date_time)].reset_index(drop= 'Frist')['open'].iloc[0]
    strike_price_ce_1 =data_ce[(data_ce['open'] < 65) & (data_ce['datetime'] == entry_date_time)].reset_index(drop= 'Frist')['strike_price'].iloc[0]
    intrday_ce_1 =data_ce[data_ce['strike_price'] == strike_price_ce_1].reset_index(drop='frist')
    intrday_ce_1.drop(index=[0],axis =0,inplace =True)
    intrday_ce_1 = intrday_ce_1.reset_index(drop = 'Frist')
    intrday_ce_1.drop(columns =['ticker','instrument_name','instrument_type','expiry_date','strike_price','date','time','open','close','low','volume','open interest', 'vix_close'],axis =1,inplace = True)
    intrday_ce_1
    data_pe = data_bn[(data_bn['instrument_type'] == 'PE') & (data_bn['expiry_date'] ==nearest_exp)|(data_bn['expiry_date'] == nearest_exp - datetime.timedelta(days=1))|(data_bn['expiry_date'] == nearest_exp - datetime.timedelta(days=2)) &  (data_bn['datetime'] > entry_date_time)].reset_index(drop='Frist')
    entry_pe_price_1 = data_pe[(data_pe['open'] < 65) & (data_pe['datetime'] == entry_date_time)].reset_index(drop= 'Frist')['open'].sort_values(ascending=False).iloc[0]
    strike_price_pe_1 =data_pe[(data_pe['open'] < 65) & (data_pe['datetime'] == entry_date_time)].reset_index(drop= 'Frist')['strike_price'].sort_values(ascending=False).iloc[0]
    intrday_pe_1 =data_pe[data_pe['strike_price'] == strike_price_pe_1].reset_index(drop='frist')
    intrday_pe_1.drop(index=[0],axis =0,inplace =True)
    intrday_pe_1 = intrday_pe_1.reset_index(drop = 'Frist')
    intrday_pe_1.drop(columns =['datetime','ticker','instrument_name','instrument_type','expiry_date','strike_price','date','time','open','close','low','volume','open interest', 'vix_close'],axis =1,inplace = True)
    #intrday_pe_1['pnl'] = 0
    intrday_ce_1['pe_high'] = intrday_pe_1['high']
    intrday_ce_1['pnl'] = 0

    #print(intrday_ce_1)

    stop_loss = 131
    ce_exit_datetime = ''
    pe_exit_datetime =''
    ce_exit_price = 0
    pe_exit_price = 0
    ce_pnl = 0
    pe_pnl = 0
    pnl = 0
    status = 0
    #re_entry_time = 
    one_minute = pd.to_timedelta('1 minute')
    #re_entry_time = trade_log['ce_exit_time'].iloc[-1] + one_minute

    for index_ce,row_ce in intrday_ce_1.iterrows():
        #print(index)
        ce_ltp = row_ce['high']
        pe_ltp = row_ce['pe_high']
        #1 Not hit stoploss any position
        if (row_ce['datetime'] == exit_date_time) & (status == 0):
            ce_pnl = entry_ce_price_1 - ce_ltp
            pe_pnl = entry_pe_price_1 - pe_ltp
            status = 1
            ce_exit_price = ce_ltp
            pe_exit_price = pe_ltp
            ce_exit_datetime = row_ce['datetime']
            pe_exit_datetime = row_ce['datetime']
            pnl = ce_pnl + pe_pnl
            print('Not hit Stop loss Call and Put')
            break
            #2 call or put hit stop loss then both are position cut same time
        elif (ce_ltp >= stop_loss) or (pe_ltp >= stop_loss) & (status == 0):
            ce_pnl = entry_ce_price_1 - ce_ltp
            pe_pnl = entry_pe_price_1 - pe_ltp
            status = 1
            ce_exit_price = ce_ltp
            pe_exit_price = pe_ltp
            ce_exit_datetime = row_ce['datetime']
            pe_exit_datetime = row_ce['datetime']
            pnl = ce_pnl + pe_pnl
            print('Stop loss hit Both Position exit')
            #continue
            break
                
    trade_log = trade_log.append({'Entry_date_time' : entry_date_time , 'Day_to_exp': (nearest_exp - entry_date_time.date()).days, 'entry_ce_price_1': entry_ce_price_1 , 'entry_pe_price_1' :entry_pe_price_1, 'entry_ce_strike_price_1':strike_price_ce_1 ,
 
                                'entry_pe_strike_price_1' :strike_price_pe_1 , 'ce_ltp': ce_exit_price , 'pe_ltp':pe_exit_price , 'ce_pnl':ce_pnl , 'pe_pnl':pe_pnl , 'ce_exit_time' :ce_exit_datetime , 'put_exit_time': pe_exit_datetime , 'pnl' : pnl },ignore_index=True) 
    
       

0
Not hit Stop loss Call and Put
1
Stop loss hit Both Position exit
2
Not hit Stop loss Call and Put
3
Not hit Stop loss Call and Put
4
Not hit Stop loss Call and Put
5
Stop loss hit Both Position exit
6
Not hit Stop loss Call and Put
7
Not hit Stop loss Call and Put
8
Not hit Stop loss Call and Put
9
Not hit Stop loss Call and Put
10
Not hit Stop loss Call and Put
11
Not hit Stop loss Call and Put
12
Stop loss hit Both Position exit
13
Not hit Stop loss Call and Put
14
Not hit Stop loss Call and Put
15
Not hit Stop loss Call and Put
16
Not hit Stop loss Call and Put
17
Not hit Stop loss Call and Put
18
Stop loss hit Both Position exit
19
Stop loss hit Both Position exit


In [6]:
trade_log

,Entry_date_time,Day_to_exp,entry_ce_price_1,entry_pe_price_1,entry_ce_strike_price_1,entry_pe_strike_price_1,ce_ltp,pe_ltp,ce_pnl,pe_pnl,ce_exit_time,put_exit_time,pnl
0,2019-01-01 09:16:00,2,55.25,52.40,27500.0,26900.0,92.80,14.65,-37.55,37.75,2019-01-01 15:20:00,2019-01-01 15:20:00,0.20
1,2019-01-02 09:16:00,1,60.30,59.90,27400.0,27100.0,135.75,34.75,-75.45,25.15,2019-01-02 10:21:00,2019-01-02 10:21:00,-50.30
2,2019-01-03 09:16:00,0,60.65,34.85,27200.0,27000.0,0.05,29.60,60.60,5.25,2019-01-03 15:20:00,2019-01-03 15:20:00,65.85
3,2019-01-04 09:16:00,6,49.85,55.30,27600.0,26500.0,61.80,20.50,-11.95,34.80,2019-01-04 15:20:00,2019-01-04 15:20:00,22.85
4,2019-01-07 09:16:00,3,44.90,54.80,27800.0,27100.0,22.05,80.00,22.85,-25.20,2019-01-07 15:20:00,2019-01-07 15:20:00,-2.35
5,2019-01-08 09:16:00,2,59.35,64.00,27500.0,27000.0,132.45,21.55,-73.10,42.45,2019-01-08 15:14:00,2019-01-08 15:14:00,-30.65
6,2019-01-09 09:16:00,1,49.40,42.75,27800.0,27400.0,45.00,20.10,4.40,22.65,2019-01-09 15:20:00,2019-01-09 15:20:00,27.05
7,2019-01-10 09:16:00,0,42.05,36.40,27700.0,27500.0,0.10,1.40,41.95,35.00,2019-01-10 15:20:00,2019-01-10 15:20:00,76.95
8,2019-01-11 09:16:00,6,43.65,58.75,28100.0,27100.0,21.65,65.20,22.00,-6.45,2019-01-11 15:20:00,2019-01-11 15:20:00,15.55
9,2019-01-14 09:16:00,3,56.35,59.15,27700.0,27000.0,38.50,58.50,17.85,0.65,2019-01-14 15:20:00,2019-01-14 15:20:00,18.50


In [9]:
do_not_entry = datetime.time(14,59)
re_exit_time = datetime.time(15,20)
re_entry_time =[]
one_minute = pd.to_timedelta('1 minute')
for time_1 in trade_log['ce_exit_time']:
    if time_1.time() < do_not_entry:
        time_1 =time_1 + one_minute
        re_entry_time.append(time_1)

In [10]:
date_list = []
time_list = []
for i in re_entry_time:
    date_list.append(i.date())
    time_list.append(i.time())
    
path_new = path[path['datetime'].isin(date_list)].reset_index(drop = 'Frist')
path_new['time'] = time_list
path_new['time'] = pd.to_datetime(path_new['time'], format='%H:%M:%S')
re_trade_log = pd.DataFrame(columns = ['re_Entry_date_time' , 're_Day_to_exp' , 're_entry_ce_price' , 're_entry_pe_price' , 're_entry_ce_strike' ,
                                   're_entry_ce_strike' , 're_ce_ltp' , 're_pe_ltp' , 're_ce_pnl' , 're_pe_pnl' , 're_ce_exit_time' , 're_put_exit_time' , 're_pnl'])

In [15]:
re_trade_log = pd.DataFrame(columns = ['re_Entry_date_time' , 're_Day_to_exp' , 're_entry_ce_price' , 're_entry_pe_price' , 're_entry_ce_strike' ,
                                   're_entry_ce_strike' , 're_ce_ltp' , 're_pe_ltp' , 're_ce_pnl' , 're_pe_pnl' , 're_ce_exit_time' ,
                                       're_put_exit_time' , 're_pnl'])

In [17]:
re_trade_log.reset_index(drop = True , inplace = True)

In [18]:
for index,row in path_new.iterrows():
    data = pd.read_pickle(row['location'])
    #print(data)
    re_entry_date_time=datetime.datetime.combine(row['datetime'].date(),row['time'].time())
    re_exit_date_time = datetime.datetime.combine(row['datetime'].date(),datetime.time(15,20))
    #break
    data=data[data['instrument_name'] == 'BANKNIFTY'].reset_index(drop = 'Frist')

    data = data[~data['instrument_type'].isin(['FUT'])]

    nearest_exp = row['datetime'].date()+relativedelta(weekday = TH)

    re_data_ce =data[(data['instrument_type'] == 'CE') & (data['expiry_date'] ==nearest_exp)|(data['expiry_date'] == nearest_exp - datetime.timedelta(days=1))|(data['expiry_date'] == nearest_exp - datetime.timedelta(days=2))].reset_index(drop='Frist')


    re_ce_price = re_data_ce[(re_data_ce['open'] < 65) & (re_data_ce['datetime'] == re_entry_date_time)]['open'].iloc[0]

    re_ce_strike =re_data_ce[(re_data_ce['open'] < 65) & (re_data_ce['datetime'] == re_entry_date_time)]['strike_price'].iloc[0]

   

    re_data_pe =data[(data['instrument_type'] == 'PE') & (data['expiry_date'] ==nearest_exp)|(data['expiry_date'] == nearest_exp - datetime.timedelta(days=1))|(data['expiry_date'] == nearest_exp - datetime.timedelta(days=2))].reset_index(drop='Frist')


    re_pe_price = re_data_pe[(re_data_pe['open'] < 65) & (re_data_pe['datetime'] == re_entry_date_time)]['open'].sort_values(ascending = False).iloc[0]

    re_pe_strike =re_data_pe[(re_data_pe['open'] < 65) & (re_data_pe['datetime'] == re_entry_date_time)]['strike_price'].sort_values(ascending=False).iloc[0]

    

    re_intrday_data_ce = re_data_ce[(re_data_ce['strike_price'] ==re_ce_strike ) &( re_data_ce['datetime'] >= re_entry_date_time)].reset_index(drop = 'Frist')
    re_intrday_data_ce.drop(['ticker','instrument_name','instrument_type','expiry_date','strike_price','date','time','open','low','close','volume','open interest','vix_close'],axis = 1,inplace = True)
    



    re_intrday_data_pe = re_data_pe[(re_data_pe['strike_price'] ==re_pe_strike ) &( re_data_pe['datetime'] >= re_entry_date_time)].reset_index(drop = 'Frist')

    re_intrday_data_pe.drop(['ticker','instrument_name','instrument_type','expiry_date','strike_price','date','time','open','datetime','low','close','volume','open interest','vix_close'],axis = 1,inplace = True)

    re_intrday_data_ce['pe_high'] = re_intrday_data_pe['high']

    re_intrday_data_ce['pnl'] = 0

    re_intrday = re_intrday_data_ce

    

    stop_loss = 131
    re_ce_exit_datetime = ''
    re_pe_exit_datetime =''
    re_ce_exit_price = 0
    re_pe_exit_price = 0
    re_ce_pnl = 0
    re_pe_pnl = 0
    re_pnl = 0
    status = 0

    for re_index,re_row in re_intrday.iterrows():
            #print(re_index)
            re_ce_ltp = re_row['high']
            re_pe_ltp = re_row['pe_high']
            #1 Not hit stoploss any position
            if (re_row['datetime'] == re_exit_date_time) & (status == 0):
                re_ce_pnl = re_ce_price - re_ce_ltp
                re_pe_pnl = re_pe_price - re_pe_ltp
                status = 1
                re_ce_exit_price = re_ce_ltp
                re_pe_exit_price = re_pe_ltp
                re_ce_exit_datetime = re_row['datetime']
                re_pe_exit_datetime = re_row['datetime']
                re_pnl = re_ce_pnl + re_pe_pnl
                print('Not hit Stop loss Call and Put')
                break
                #2 call or put hit stop loss then both are position cut same time
            elif (re_ce_ltp >= stop_loss) or (re_pe_ltp >= stop_loss) & (status == 0):
                re_ce_pnl = re_ce_price - re_ce_ltp
                re_pe_pnl = re_pe_price - re_pe_ltp
                status = 1
                re_ce_exit_price = re_ce_ltp
                re_pe_exit_price = re_pe_ltp
                re_ce_exit_datetime = re_row['datetime']
                re_pe_exit_datetime = re_row['datetime']
                re_pnl = re_ce_pnl + re_pe_pnl
                print('Stop loss hit Both Position exit')
                #continue
                break


    re_trade_log = re_trade_log.append({'re_Entry_date_time' : re_entry_date_time , 're_Day_to_exp': (nearest_exp - re_entry_date_time.date()).days , 're_entry_ce_price':re_ce_price , 're_entry_pe_price':re_pe_price , 're_entry_ce_strike':re_ce_strike ,
                                       're_entry_ce_strike':re_pe_strike , 're_ce_ltp': re_ce_exit_price , 're_pe_ltp':re_pe_exit_price , 're_ce_pnl':re_ce_pnl , 're_pe_pnl':re_pe_pnl , 're_ce_exit_time':re_ce_exit_datetime , 're_put_exit_time':re_pe_exit_datetime , 're_pnl':re_pnl},ignore_index=True)
        

Stop loss hit Both Position exit


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
re_trade_log